In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, count, avg, min, max, stddev, expr

# Initialize Spark Session
spark = SparkSession.builder.appName('AnimeRatingsPipeline').getOrCreate()

## Load Data

In [5]:
df = spark.read.csv('users-score-2023.csv', header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- user_id: integer (nullable = true)
 |-- Username: string (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- Anime Title: string (nullable = true)
 |-- rating: string (nullable = true)

+-------+--------+--------+--------------------+------+
|user_id|Username|anime_id|         Anime Title|rating|
+-------+--------+--------+--------------------+------+
|      1|   Xinil|      21|           One Piece|     9|
|      1|   Xinil|      48|         .hack//Sign|     7|
|      1|   Xinil|     320|              A Kite|     5|
|      1|   Xinil|      49|    Aa! Megami-sama!|     8|
|      1|   Xinil|     304|Aa! Megami-sama! ...|     8|
+-------+--------+--------+--------------------+------+
only showing top 5 rows


## Apply Transformations
### *1. 2+filter operations*

In [6]:
# Filter 1: Keep ratings >= 5
df_filtered = df.filter(expr("try_cast(rating as double) >= 5"))

# Filter 2: Exclude users with fewer than 10 ratings
user_counts = df_filtered.groupBy('user_id').count()
active_users = user_counts.filter(col('count') >= 10)

# Join back to keep only active users
df_filtered = df_filtered.join(active_users, on='user_id', how='inner')
df_filtered.show(5)

+-------+--------+--------+--------------------+------+-----+
|user_id|Username|anime_id|         Anime Title|rating|count|
+-------+--------+--------+--------------------+------+-----+
|   1342|  hdphnz|     306|Abenobashi Mahou☆...|     7|  181|
|   1342|  hdphnz|    1051|         Ao no 6-gou|     8|  181|
|   1342|  hdphnz|     492|Armitage III: Dua...|     9|  181|
|   1342|  hdphnz|    2986|        Bamboo Blade|     8|  181|
|   1342|  hdphnz|      57|                Beck|    10|  181|
+-------+--------+--------+--------------------+------+-----+
only showing top 5 rows


### *2. 1+complex aggregation*

In [7]:
#multiple summary statistics per anime
complex_agg_df = df_filtered.groupBy("anime_id", "Anime Title").agg(
    avg("rating").alias("avg_rating"),
    expr("percentile_approx(rating, 0.5)").alias("median_rating"),
    min("rating").alias("min_rating"),
    max("rating").alias("max_rating"),
    stddev("rating").alias("stddev_rating"),
    count("*").alias("num_ratings")
)

complex_agg_df.show(10)

+--------+--------------------+-----------------+-------------+----------+----------+------------------+-----------+
|anime_id|         Anime Title|       avg_rating|median_rating|min_rating|max_rating|     stddev_rating|num_ratings|
+--------+--------------------+-----------------+-------------+----------+----------+------------------+-----------+
|      44|Rurouni Kenshin: ...|8.920852732173486|          9.0|        10|         9| 1.150913506632057|      20405|
|      50|Aa! Megami-sama! ...|7.575025536261491|          8.0|        10|         9|1.3230211555949083|      19580|
|      72|Full Metal Panic?...|8.379640968187385|          8.0|        10|         9|1.2323295201300628|      38381|
|      73|Full Metal Panic!...|8.197916350271846|          8.0|        10|         9|1.1627792907197914|      32923|
|      74|        Gakuen Alice| 7.88901549680948|          8.0|        10|         9|1.3583927612774136|       8776|
|      91|Shin Kidou Senki ...|7.996696463272444|          8.0| 

### *3. 1+groupBy with aggredations*

In [8]:
# Compute average rating and number of ratings per anime
agg_df = df_filtered.groupBy('anime_id', 'Anime Title').agg(
    avg('rating').alias('avg_rating'),
    count('*').alias('num_ratings')
)

agg_df.show(10)

+--------+--------------------+------------------+-----------+
|anime_id|         Anime Title|        avg_rating|num_ratings|
+--------+--------------------+------------------+-----------+
|     904|Dragon Ball Z Mov...|6.9268725070172845|       6769|
|   19111|Love Live! School...| 7.936881969282559|       4753|
|     526|Boku no Chikyuu w...| 7.444284687275341|       1391|
|     323|     Mousou Dairinin| 7.924207873259721|      16664|
|    4705|Tengen Toppa Gurr...| 7.554706601466993|       3272|
|    5332|Toshokan Sensou: ...|7.6279547790339155|        973|
|    4918|   xxxHOLiC Shunmuki|   8.2503885003885|       5148|
|    3359|            Amatsuki| 7.238871320846509|       8222|
|      74|        Gakuen Alice|  7.88901549680948|       8776|
|   32086|        Blame! Movie| 7.458149779735683|       3178|
+--------+--------------------+------------------+-----------+
only showing top 10 rows


### *4. Column transformation*

In [9]:
# Round average ratings and create popularity metric
agg_df = agg_df.withColumn('avg_rating', round(col('avg_rating'), 2))
agg_df = agg_df.withColumn('popularity', col('avg_rating') * col('num_ratings'))

agg_df.show(10)

+--------+--------------------+----------+-----------+----------+
|anime_id|         Anime Title|avg_rating|num_ratings|popularity|
+--------+--------------------+----------+-----------+----------+
|     904|Dragon Ball Z Mov...|      6.93|       6769|  46909.17|
|   19111|Love Live! School...|      7.94|       4753|  37738.82|
|     526|Boku no Chikyuu w...|      7.44|       1391|  10349.04|
|     323|     Mousou Dairinin|      7.92|      16664| 131978.88|
|    4705|Tengen Toppa Gurr...|      7.55|       3272|   24703.6|
|    5332|Toshokan Sensou: ...|      7.63|        973|   7423.99|
|    4918|   xxxHOLiC Shunmuki|      8.25|       5148|   42471.0|
|    3359|            Amatsuki|      7.24|       8222|  59527.28|
|      74|        Gakuen Alice|      7.89|       8776|  69242.64|
|   32086|        Blame! Movie|      7.46|       3178|  23707.88|
+--------+--------------------+----------+-----------+----------+
only showing top 10 rows


## 2+SQL queries

In [10]:
# Register temporary view
agg_df.createOrReplaceTempView('anime_stats')

# Query 1: Top 10 anime by average rating (with min ratings > 500)
top_avg = spark.sql('''
SELECT `Anime Title`, avg_rating, num_ratings
FROM anime_stats
WHERE num_ratings > 500
ORDER BY avg_rating DESC
LIMIT 10
''')
top_avg.show()

# Query 2: Top 10 anime by popularity
top_pop = spark.sql('''
SELECT `Anime Title`, popularity, num_ratings
FROM anime_stats
ORDER BY popularity DESC
LIMIT 10
''')
top_pop.show()

+--------------------+----------+-----------+
|         Anime Title|avg_rating|num_ratings|
+--------------------+----------+-----------+
|            Gintama°|      9.21|       6097|
|Fullmetal Alchemi...|      9.18|      72889|
|            Gintama'|      9.18|      12424|
|Ginga Eiyuu Densetsu|      9.15|       6056|
| Gintama': Enchousen|      9.11|       7586|
|Clannad: After Story|       9.1|      49600|
|         Steins;Gate|      9.06|      41048|
|Gintama Movie 2: ...|      9.05|       6509|
|      Kimi no Na wa.|       9.0|      23631|
|Code Geass: Hangy...|      8.96|      78442|
+--------------------+----------+-----------+



+--------------------+-----------------+-----------+
|         Anime Title|       popularity|num_ratings|
+--------------------+-----------------+-----------+
|          Death Note|       1013304.24|     115674|
|Code Geass: Hangy...|        813340.13|      92111|
| Fullmetal Alchemist|        741966.45|      88015|
|Code Geass: Hangy...|702840.3200000001|      78442|
|Fullmetal Alchemi...|        669121.02|      72889|
|          Elfen Lied|651618.7999999999|      79660|
|              Bleach|616105.2000000001|      77988|
|              Naruto|         598966.5|      77286|
|Sen to Chihiro no...|         595312.1|      66889|
|        Angel Beats!|        566665.45|      67061|
+--------------------+-----------------+-----------+



## Optimatizon of queries

In [12]:

# Push filters early and cache intermediate results
spark.sql("""
CREATE OR REPLACE TEMP VIEW filtered_anime AS
SELECT *
FROM anime_stats
WHERE num_ratings > 500
""")

# Repartition by Anime Title to reduce shuffle during ORDER BY
filtered_df = spark.table("filtered_anime").repartition(8, "Anime Title")
filtered_df.cache()
filtered_df.createOrReplaceTempView("filtered_anime_partitioned")

# Optimized Query 1: Top 10 anime by average rating
top_avg_opt = spark.sql("""
SELECT `Anime Title`, avg_rating, num_ratings
FROM filtered_anime_partitioned
ORDER BY avg_rating DESC
LIMIT 10
""")
print("✅ Optimized Query 1: Top 10 Anime by Average Rating")
top_avg_opt.show()

# Optimized Query 2: Top 10 anime by popularity (same cached subset reused)
top_pop_opt = spark.sql("""
SELECT `Anime Title`, popularity, num_ratings
FROM filtered_anime_partitioned
ORDER BY popularity DESC
LIMIT 10
""")
print("✅ Optimized Query 2: Top 10 Anime by Popularity")
top_pop_opt.show()


base_path = "output/anime_results"
top_avg_opt.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/top_avg_anime.parquet")
top_pop_opt.coalesce(1).write.mode("overwrite").parquet(f"{base_path}/top_pop_anime.parquet")


✅ Optimized Query 1: Top 10 Anime by Average Rating


+--------------------+----------+-----------+
|         Anime Title|avg_rating|num_ratings|
+--------------------+----------+-----------+
|            Gintama°|      9.21|       6097|
|            Gintama'|      9.18|      12424|
|Fullmetal Alchemi...|      9.18|      72889|
|Ginga Eiyuu Densetsu|      9.15|       6056|
| Gintama': Enchousen|      9.11|       7586|
|Clannad: After Story|       9.1|      49600|
|         Steins;Gate|      9.06|      41048|
|Gintama Movie 2: ...|      9.05|       6509|
|      Kimi no Na wa.|       9.0|      23631|
|Code Geass: Hangy...|      8.96|      78442|
+--------------------+----------+-----------+

✅ Optimized Query 2: Top 10 Anime by Popularity
+--------------------+-----------------+-----------+
|         Anime Title|       popularity|num_ratings|
+--------------------+-----------------+-----------+
|          Death Note|       1013304.24|     115674|
|Code Geass: Hangy...|        813340.13|      92111|
| Fullmetal Alchemist|        741966.45|  